In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc ,classification_report,roc_auc_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from gensim.models.word2vec import Word2Vec
from zhon.hanzi import punctuation
import numpy as np
import pandas as pd
import jieba
from sklearn.externals import joblib
from sklearn.svm import SVC
import sys 
import re

In [2]:
dir_1 = r'all_data_new_1.csv'

df = pd.read_csv(dir_1)
print(df.shape)
df.head()

(16251, 3)


,text,title,type
0,"They stood in line at Trump Tower, sometimes u...","At Donald Trump<U+2019>s Properties, a Showcas...",1
1,Donald J. Trump <U+2019>s foundation informed ...,Trump Foundation Tells New York It Has Stopped...,1
2,President-elect Donald J. Trump won the White ...,"Donald Trump Prepares for White House Move, bu...",1
3,An investment pitch for a new Texas hotel is t...,Luring Chinese Investors With Trump<U+2019>s N...,1
4,President-elect Donald J. Trump <U+2019>s wife...,Melania and Barron Trump Won<U+2019>t Immediat...,1


In [3]:
def cleanlines(line,remove_stopwords=False):   
    p1=re.compile(r'-\{.*?(zh-hans|zh-cn):([^;]*?)(;.*?)?\}-')
    p2=re.compile(r'[(][: @ . , ？！\s][)]')
    p3=re.compile(r'[「『]')
    p4=re.compile(r'[\s+\.\!\/_,$%^*(+\"\')]+|[+——()?【】“”！，。？、~@#￥%……&*（）0-9 , : ; \-\ \[\ \]\ ]')
    p5=re.compile(r'[^a-zA-Z]')
    line=p1.sub(r' ',line)
    line=p2.sub(r' ',line)
    line=p3.sub(r' ',line)
    line=p4.sub(r' ',line)
    line=p5.sub(r' ',line)
    words = line.lower().split()
    if remove_stopwords:
        words = [w for w in words if w not in eng_stopwords]
    return words

In [4]:
eng_stopwords = [line.rstrip() for line in open('stop_words.txt')]
eng_stopwords[:5]

["'d", "'ll", "'m", "'re", "'s"]

In [7]:
df['text'] = df['text'].astype(str)
sents = df['text']
text_words=[cleanlines(sent,remove_stopwords=True) for sent in sents]

In [6]:
df['title'] = df['title'].astype(str)
title_sents = df['title']
title_words=[cleanlines(sent,remove_stopwords=True) for sent in title_sents]

In [8]:
def concat_text(text_words,title_words):
    concat_text = []
    for i in range(len(text_words)):
        temp = text_words[i]+title_words[i]
        concat_text.append(temp)
    return concat_text

concat_text = concat_text(text_words,title_words)

In [8]:
def build_sentence_vector(text, size,imdb_w2v):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += imdb_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [41]:
#word2vec
def get_train_vecs(x_samples):
    n_dim = 300
    #initialize model and dict
    rj_w2v = Word2Vec(size=n_dim, min_count=5)
    rj_w2v.build_vocab(x_samples)
    
    #train
    rj_w2v.train(x_samples,total_examples=rj_w2v.corpus_count, epochs=300)
    
    all_vecs = np.concatenate([build_sentence_vector(z, n_dim,rj_w2v) for z in x_samples])
    #train_vecs = scale(train_vecs)
    
    np.save('x_samples2.npy',all_vecs)
    print(all_vecs.shape)


In [9]:
get_train_vecs(concat_text)

NameError: name 'get_train_vecs' is not defined

In [10]:
def get_data():
    train_vecs=np.load('x_samples2.npy')
    return train_vecs
x_samples_arr = get_data()

In [15]:
x_samples_arr.shape
x_samples_arr

array([[ 2.59894514e-04, -8.80054816e-03, -3.68411109e-01, ...,
         6.43377418e-01,  1.12992524e-02,  1.46089972e-01],
       [-1.29897315e-01,  4.34215091e-01, -2.38818198e-02, ...,
        -3.65664936e-01, -3.32856275e-01,  5.67314321e-02],
       [-7.22000141e-02,  8.86973100e-02, -7.18838336e-01, ...,
         6.88309858e-01, -7.65332874e-02,  1.85850647e-01],
       ...,
       [ 5.33787489e-01,  1.51734551e-02, -5.83059947e-01, ...,
         1.45384519e+00, -3.93676023e-01, -5.22272954e-01],
       [ 9.45472856e-01,  5.97023793e-02,  7.16998306e-01, ...,
         4.00667506e-01,  1.69707319e+00, -2.26624767e-01],
       [ 2.12376118e-01,  7.69252807e-01,  2.04288626e+00, ...,
         9.32398915e-01,  3.12417172e-01,  9.32964072e-01]])

In [16]:
y = np.array(df['type']).astype(int)
print(len(y))

16251


In [17]:

x_train,x_val,y_train,y_val = train_test_split(x_samples_arr,y,test_size=0.2,random_state=10,stratify=y)


In [18]:
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)
y_train = y_train.reshape(13000,1)
print(y_train.shape)


(13000, 300)
(3251, 300)
(13000,)
(3251,)
(13000, 1)


In [19]:
from tensorflow.contrib import learn
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D,Input, concatenate
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape

Using TensorFlow backend.


In [20]:
vocab_size = 5000 #dict size
batch_size = 200 #batch size for each training
sequence_length = 300 #sequence length -same for every sample
embedding_dims=128 #embed dim

In [30]:
#Inputs:Input函数只需要填写列数
comment_seq = Input(shape=[sequence_length],name='x_seq')  #shape is (number of samples eg:10000,300) 
print(comment_seq.shape)
#embedding layers
embedding_layer = Embedding(1, embedding_dims)(comment_seq)   #填充层，Embedding(len(word_to_id), embedding_dims) 内置 5000(len word_to_id) 0....4999*128 random vector
# comment_seq 放入变成 10000*300*128
print(embedding_layer.shape)

(?, 300)
(?, 300, 128)


In [31]:
#在原来的基础上添加一个维度，相当于 类似图片数据(number sequence,r,g,b) 二维精度高 考虑了前后
'''Reshape Layer'''
reshape = Reshape(target_shape=(sequence_length, embedding_dims, 1), name='reshape')(embedding_layer) # chanels last
print(reshape.shape)
print("Builded Reshape Layer...")

(?, 300, 128, 1)
Builded Reshape Layer...


In [32]:
#使用三种filter进行过滤，将最后的结果flatten  text 只能上下移动 filter
nb_filters = 100   
filter_lengths = [3, 4, 5]   # 3*128 , 4*128, 5*128
flatten_ = []

for i in filter_lengths:
    model_internal = Sequential()
    model_internal.add(Convolution2D(nb_filters, (i, embedding_dims), activation="relu", name='conv2d_' + str(i), input_shape=(sequence_length, embedding_dims, 1)))
# chanels last,默认了strides=(1,1), padding='valid'
    model_internal.add(MaxPooling2D(pool_size=(sequence_length - i + 1, 1), name='maxpool2d_' + str(i)))
    model_internal.add(Flatten())   #flatten 去除刚刚加的最后的一纬
    flatten = model_internal(reshape)  
    #print(flatten.shape)
    flatten_.append(flatten)   #list of 三个矩阵

In [33]:
#Fully Connect Layer & Dropout Layer
dropout_rate = 0.5   
hidden_nodes = 256   
merge = concatenate(flatten_, axis=-1)    

fully_connect = Dense(hidden_nodes, activation='relu', name='fully_connect')(merge) 
dropout = Dropout(dropout_rate, name='dropout')(fully_connect)

print("Builded Fully Connect Layer & Dropout Layer...")

'''Projection Layer & Output Layer'''

output = Dense(1, activation='sigmoid', name='output')(dropout) # output layer

print("Builded  Output Layer...")

Builded Fully Connect Layer & Dropout Layer...
Builded  Output Layer...


In [34]:
from keras.models import Model


In [35]:
model = Model([comment_seq],output)

In [36]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy']) #binary classification

In [37]:
from keras.callbacks import EarlyStopping,ModelCheckpoint

es = EarlyStopping(monitor='val_acc',patience=5)  #val patience times

filepath="weights.bests.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,mode='max')
callbacks_list = [es,checkpoint]

In [38]:
model.fit(x_train, y_train,
          validation_split=0.1,
          batch_size=batch_size,   
          callbacks=[es],
          epochs=5,
          shuffle=True)

Train on 11700 samples, validate on 1300 samples
Epoch 1/5


InvalidArgumentError: indices[138,4] = -1 is not in [0, 1)
	 [[{{node embedding_2/embedding_lookup}} = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@training_1/Adam/Assign_2"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_2/embeddings/read, embedding_2/Cast, training_1/Adam/gradients/embedding_2/embedding_lookup_grad/concat/axis)]]